# Base Sepolia Transaction Monitor

Minimal notebook to connect to Base Sepolia explorer and analyze transaction completion times.

In [1]:
import requests
import json
from datetime import datetime

In [2]:
# Base Sepolia Blockscout API configuration
BASE_URL = "https://base-sepolia.blockscout.com/api/v2"

def get_transaction_details(tx_hash: str):
    """Get transaction details from Base Sepolia Blockscout"""
    url = f"{BASE_URL}/transactions/{tx_hash}"
    
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"API Error: {response.status_code} - {response.text}")
    return None

In [3]:
def analyze_transaction(tx_hash: str):
    """Analyze a specific transaction"""
    print(f"Analyzing: {tx_hash}")
    
    tx_data = get_transaction_details(tx_hash)
    if not tx_data:
        print("Transaction not found")
        return None
    
    # Extract key data
    block_number = tx_data.get('block_number')
    timestamp = tx_data.get('timestamp')
    confirmation_duration = tx_data.get('confirmation_duration', [])
    gas_used = int(tx_data.get('gas_used', 0))
    gas_price = int(tx_data.get('gas_price', 0))
    
    # Calculate fees
    actual_fee = tx_data.get('fee', {})
    if actual_fee and 'value' in actual_fee:
        actual_fee_eth = int(actual_fee['value']) / 1e18
    else:
        actual_fee_eth = (gas_used * gas_price) / 1e18
    
    status = tx_data.get('status', 'unknown')
    method = tx_data.get('method', 'N/A')
    from_address = tx_data.get('from', {}).get('hash', 'N/A')
    to_address = tx_data.get('to', {}).get('hash', 'N/A')
    
    # Print summary
    print(f"Block: {block_number}")
    print(f"Time: {timestamp}")
    print(f"Status: {status}")
    print(f"Method: {method}")
    print(f"Gas: {gas_used:,}")
    print(f"Fee: {actual_fee_eth:.6f} ETH")
    
    return {
        'hash': tx_hash,
        'block': block_number,
        'timestamp': timestamp,
        'confirmation_duration_ms': confirmation_duration[1] if confirmation_duration else None,
        'status': status,
        'method': method,
        'gas_used': gas_used,
        'actual_fee_eth': actual_fee_eth,
        'from': from_address,
        'to': to_address
    }

def get_transaction_logs(tx_hash: str):
    """Get transaction logs to find Oracle request ID"""
    url = f"{BASE_URL}/transactions/{tx_hash}/logs"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get('items', [])
    else:
        print(f"API Error: {response.status_code} - {response.text}")
    return None

In [4]:
# COMPLETE ORACLE REQUEST-FULFILLMENT ANALYSIS (FIXED DELAY BREAKDOWN)
def analyze_oracle_request_fulfillment(request_tx_hash, fulfillment_tx_hash=None):
    """Complete analysis of Oracle request and fulfillment with correct timing breakdown."""
    
    from datetime import datetime
    
    print("="*80)
    print("ORACLE REQUEST-FULFILLMENT ANALYSIS")
    print("="*80)
    
    # STEP 1: REQUEST TRANSACTION
    print("STEP 1: REQUEST TRANSACTION ANALYSIS")
    print("-" * 50)
    
    request_result = analyze_transaction(request_tx_hash)
    if not request_result:
        print("Failed to analyze request transaction")
        return None
    
    # Raw request transaction info
    print(f"Analyzing: {request_tx_hash}")
    print(f"Block: {request_result['block']}")
    print(f"Time: {request_result['timestamp']}")
    print(f"Status: {request_result.get('status', 'unknown')}")
    print(f"Method: {request_result.get('method', 'unknown')}")
    print(f"Gas: {request_result.get('gas_used', 'N/A')}")
    print(f"Fee: {request_result.get('fee_eth', 'N/A')} ETH")
    
    # Extract logs for OracleRequest
    request_logs = get_transaction_logs(request_tx_hash)
    
    spec_id = None
    request_id = None
    
    ORACLE_REQUEST_TOPIC = "0xd8d7ecc4800d25fa53ce0372f13a416d98907a7ef3d8d3bdd79cf4fe75529c65"
    
    for log in request_logs:
        topics = log.get("topics", [])
        if topics and topics[0].lower() == ORACLE_REQUEST_TOPIC.lower():
            spec_id = topics[1]
            data = log.get("data", "")
            # requestId is bytes32 in the data payload (2nd slot → bytes 32..64)
            if len(data) >= 130:  # "0x" + 64 hex per word * 2 words
                request_id = "0x" + data[66:130]
            break
    
    print(f"Spec ID (job):      {spec_id}")
    print(f"Request ID:         {request_id}")
    print(f"Request Time:       {request_result['timestamp']}")
    print(f"Request Block:      {request_result['block']}")
    
    # STEP 2: FULFILLMENT
    if fulfillment_tx_hash:
        print("\nSTEP 2: FULFILLMENT TRANSACTION ANALYSIS")
        print("-" * 50)
        
        fulfillment_result = analyze_transaction(fulfillment_tx_hash)
        if not fulfillment_result:
            print("Failed to analyze fulfillment transaction")
            return None
        
        print(f"Analyzing: {fulfillment_tx_hash}")
        print(f"Block: {fulfillment_result['block']}")
        print(f"Time: {fulfillment_result['timestamp']}")
        print(f"Status: {fulfillment_result.get('status', 'unknown')}")
        print(f"Method: {fulfillment_result.get('method', 'unknown')}")
        print(f"Gas: {fulfillment_result.get('gas_used', 'N/A')}")
        print(f"Fee: {fulfillment_result.get('fee_eth', 'N/A')} ETH")
        print(f"Fulfillment Time:   {fulfillment_result['timestamp']}")
        print(f"Fulfillment Block:  {fulfillment_result['block']}")
        
        # --- TIMING ANALYSIS ---
        print("\nTIMING ANALYSIS:")
        
        # 1) End-to-end (block time → block time)
        request_dt = datetime.fromisoformat(request_result['timestamp'].replace('Z', '+00:00'))
        fulfillment_dt = datetime.fromisoformat(fulfillment_result['timestamp'].replace('Z', '+00:00'))
        
        total_response_time_ms = (fulfillment_dt - request_dt).total_seconds() * 1000
        print(f"Total End-to-End Time: {total_response_time_ms:.0f} ms ({total_response_time_ms/1000:.2f} seconds)")
        
        # 2) Oracle internal processing (from your node logs)
        #    These are specific to THIS request, hard-coded from the JobRun/taskRuns
        oracle_created = datetime.fromisoformat("2025-12-12T15:39:52.223925+00:00")
        oracle_processing_done = datetime.fromisoformat("2025-12-12T15:39:52.980783+00:00")
        oracle_processing_time_ms = (oracle_processing_done - oracle_created).total_seconds() * 1000
        
        print(f"Oracle Processing Time (JobRun → encode_tx): {oracle_processing_time_ms:.0f} ms")
        
        # 3) Oracle detection delay: block time → JobRun.createdAt
        oracle_detection_time_ms = (oracle_created - request_dt).total_seconds() * 1000
        print(f"Oracle Detection Delay (Block → JobRun):     {oracle_detection_time_ms:.0f} ms")
        
        # 4) External adapter time (from fetch step in logs)
        external_adapter_time_ms = 740  # from fetch step measurement (15:39:52.240207Z to 15:39:52.979846Z)
        print(f"External Adapter Time (HTTP fetch):          {external_adapter_time_ms} ms")
        
        # 5) Callback mining delay: encode_tx finished → fulfillment block timestamp
        #    end_to_end = detection + oracle_internal + callback_mining
        callback_mining_delay_ms = total_response_time_ms - oracle_detection_time_ms - oracle_processing_time_ms
        print(f"Callback Mining Delay (Oracle → Fulfillment): {callback_mining_delay_ms:.0f} ms")
        
        # 6) Aggregate "non-oracle" delay (detection + mining)
        total_non_oracle_delay_ms = oracle_detection_time_ms + callback_mining_delay_ms
        print(f"Total Non-Oracle Delay (Detection + Mining): {total_non_oracle_delay_ms:.0f} ms")
        
        # --- Fulfillment Request ID check ---
        print("\nFulfillment Request ID Check:")
        fulfillment_logs = get_transaction_logs(fulfillment_tx_hash)
        request_id_found = False
        
        if fulfillment_logs and request_id:
            for log in fulfillment_logs:
                for topic in log.get('topics', []):
                    if topic.lower() == request_id.lower():
                        request_id_found = True
                        break
                if request_id_found:
                    break
        
        if request_id_found:
            print("✔ Fulfillment contains matching Request ID")
        else:
            print("❌ Warning: Fulfillment does NOT contain matching Request ID")
        
        return {
            'request_tx': request_tx_hash,
            'fulfillment_tx': fulfillment_tx_hash,
            'request_time': request_result['timestamp'],
            'fulfillment_time': fulfillment_result['timestamp'],
            'total_response_time_ms': total_response_time_ms,
            'total_response_time_seconds': total_response_time_ms / 1000,
            'oracle_processing_time_ms': oracle_processing_time_ms,
            'oracle_detection_time_ms': oracle_detection_time_ms,
            'callback_mining_delay_ms': callback_mining_delay_ms,
            'total_non_oracle_delay_ms': total_non_oracle_delay_ms,
            'request_id': request_id,
            'spec_id': spec_id
        }
    
    else:
        print("\nNo fulfillment transaction provided")
        print(f"To find fulfillment, search for transactions containing request ID: {request_id}")
    
    return {
        'request_tx': request_tx_hash,
        'request_time': request_result['timestamp'],
        'request_id': request_id,
        'spec_id': spec_id
    }


# CURRENT ANALYSIS
# Set your transaction hashes here
REQUEST_TX = "0x0d66d80ad20366bea63df62710f018ec4e22a2e8a22d371d9253fae4e80b2483"
FULFILLMENT_TX = "0xdd152db173ae8fa80c23e71e93969945ace0939f338f3e10457c62fabb014952"

# Run complete analysis
result = analyze_oracle_request_fulfillment(REQUEST_TX, FULFILLMENT_TX)

print(f"\nFINAL RESULTS:")
print("="*50)
if result and 'oracle_processing_time_ms' in result:
    print(f"Oracle Detection Delay:   {result['oracle_detection_time_ms']:.0f} ms")
    print(f"Oracle Processing Time:   {result['oracle_processing_time_ms']:.0f} ms")
    print(f"Callback Mining Delay:    {result['callback_mining_delay_ms']:.0f} ms")
    print(f"Total Non-Oracle Delay:   {result['total_non_oracle_delay_ms']:.0f} ms")
    print(f"Total End-to-End Time:    {result['total_response_time_ms']:.0f} ms "
          f"({result['total_response_time_seconds']:.2f} seconds)")
    print(f"Request Transaction:      {result['request_tx']}")
    print(f"Fulfillment Transaction:  {result['fulfillment_tx']}")
    print(f"Status: COMPLETE")
else:
    print(f"Request Transaction: {REQUEST_TX}")
    print(f"Status: PENDING FULFILLMENT")

print("="*80)


ORACLE REQUEST-FULFILLMENT ANALYSIS
STEP 1: REQUEST TRANSACTION ANALYSIS
--------------------------------------------------
Analyzing: 0x0d66d80ad20366bea63df62710f018ec4e22a2e8a22d371d9253fae4e80b2483
Block: 34892852
Time: 2025-12-12T15:39:52.000000Z
Status: ok
Method: 0x42cac024
Gas: 191,916
Fee: 0.000019 ETH
Analyzing: 0x0d66d80ad20366bea63df62710f018ec4e22a2e8a22d371d9253fae4e80b2483
Block: 34892852
Time: 2025-12-12T15:39:52.000000Z
Status: ok
Method: 0x42cac024
Gas: 191916
Fee: N/A ETH
Spec ID (job):      0x97e7b8cfa30646e4bcdbbf9a60ae1d7400000000000000000000000000000000
Request ID:         0xfce36198fba7690744953d5daca132d577c8e16b16a51adb08028a6045b0257d
Request Time:       2025-12-12T15:39:52.000000Z
Request Block:      34892852

STEP 2: FULFILLMENT TRANSACTION ANALYSIS
--------------------------------------------------
Analyzing: 0xdd152db173ae8fa80c23e71e93969945ace0939f338f3e10457c62fabb014952
Block: 34892853
Time: 2025-12-12T15:39:54.000000Z
Status: ok
Method: fulfillOracle